In [1]:
import time

import pandas as pd
import numpy as np
from selenium import webdriver

In [2]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--window-size=1920,1080")
chrome_options.add_argument("--start-maximized")

In [3]:
import os
os.getcwd()

'/Users/stevengeorge/Documents/Github/fpl-analysis/notebooks/DeepFantasyFootball'

In [4]:
driver = webdriver.Chrome(
    "../../../../Python/Chrome Driver/chromedriver",
    options=chrome_options,
)

In [5]:
driver.implicitly_wait(10)

In [6]:
TABLE_XPATH = '//*[@id="tournamentTable"]'

In [7]:
COLUMN_NAMES = ['KO', 'Match', 'Result', '1', 'X', '2', 'num_available_bookmakers']

In [8]:
ALL_SEASONS = [f'20{i}-20{i+1}' for i in range(11, 20)]

In [9]:
%%time
all_season_data = pd.DataFrame()

for season in ALL_SEASONS:
    
    print(season)

    season_odds_df = pd.DataFrame()

    for page_number in range(1, 9):
        print(page_number)

        if season == '2019-2020':
            driver.get(f'https://www.oddsportal.com/soccer/england/premier-league/results/#/page/{page_number}/')
        else:
            driver.get(f'https://www.oddsportal.com/soccer/england/premier-league-{season}/results/#/page/{page_number}/')
        
        time.sleep(1)

        tbl = driver.find_element_by_xpath(TABLE_XPATH).get_attribute("outerHTML")
        odds_table = pd.read_html(tbl, header=0)[0]

        odds_table.columns = COLUMN_NAMES
        odds_table.dropna(axis=0, how='all', inplace=True)

        # Keep matches only (some rows are repeats of the header)
        odds_table = odds_table[odds_table['Match'].str.contains('-')]

        for odd_col in ['1', 'X', '2']:
            odds_table[odd_col].replace('-', np.nan, inplace=True)
            odds_table.loc[
                ~odds_table[odd_col].isnull(), 
                odd_col
            ] = odds_table.loc[
                ~odds_table[odd_col].isnull(), 
                odd_col
            ].str.split('/').apply(lambda x: float(x[0]) / float(x[1]))

        odds_table['home_team'] = odds_table['Match'].str.split(' - ').apply(lambda x: x[0])
        odds_table['away_team'] = odds_table['Match'].str.split(' - ').apply(lambda x: x[1])

        print(odds_table.shape)

        season_odds_df = season_odds_df.append(odds_table)

    assert season_odds_df.shape[0] == 380

    season_odds_df['season'] = season
    
    all_season_data = all_season_data.append(season_odds_df)

2011-2012
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2012-2013
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2013-2014
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2014-2015
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2015-2016
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2016-2017
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2017-2018
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2018-2019
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
2019-2020
1
(50, 9)
2
(50, 9)
3
(50, 9)
4
(50, 9)
5
(50, 9)
6
(50, 9)
7
(50, 9)
8
(30, 9)
CPU times: user 2.72 s, sys: 144 ms, total: 2.87 s
Wall time: 1min 27s


In [10]:
all_season_data.shape

(3420, 10)

In [11]:
380*9

3420

In [12]:
all_season_data.head()

,KO,Match,Result,1,X,2,num_available_bookmakers,home_team,away_team,season
2,14:00,Chelsea - Blackburn,2:1,0.25,5.05,9.87,6,Chelsea,Blackburn,2011-2012
3,14:00,Everton - Newcastle,3:1,1.03,2.64,2.45,6,Everton,Newcastle,2011-2012
4,14:00,Manchester City - QPR,3:2,0.12,7.76,18.06,6,Manchester City,QPR,2011-2012
5,14:00,Norwich - Aston Villa,2:0,1.02,2.46,2.65,6,Norwich,Aston Villa,2011-2012
6,14:00,Stoke - Bolton,2:2,1.15,2.86,1.99,6,Stoke,Bolton,2011-2012


In [13]:
all_season_data[all_season_data['1'].isnull()]['season'].unique()

array(['2015-2016'], dtype=object)

In [14]:
all_season_data.isnull().sum()

KO                           0
Match                        0
Result                       0
1                           28
X                           28
2                           28
num_available_bookmakers     0
home_team                    0
away_team                    0
season                       0
dtype: int64

In [27]:
all_season_data.sample(n=5)

,KO,Match,Result,1,X,2,num_available_bookmakers,home_team,away_team,season
15,17:00,Manchester Utd - West Ham,1:1,0.26,5.55,10.43,12,Manchester Utd,West Ham,2019-2020
52,16:15,West Ham - Manchester Utd,0:2,5.34,3.42,0.54,8,West Ham,Manchester Utd,2016-2017
52,15:00,Tottenham - Manchester City,0:1,2.35,2.62,1.18,8,Tottenham,Manchester City,2014-2015
4,14:00,West Ham - Wigan,2:0,1.38,2.37,2.06,7,West Ham,Wigan,2012-2013
5,19:00,Chelsea - Tottenham,2:0,1.38,2.26,2.28,11,Chelsea,Tottenham,2018-2019


In [15]:
driver.close()

In [17]:
all_season_data.to_parquet('../../data/external/oddsportal_odds_2011to2020.parquet', index=False)